# Training: Python and GOES-R Imagery: Projections

Used this link as example to merge the GOES data for the Tonga eruption
https://geonetcast.wordpress.com/2019/08/02/goes-16-goes-17-plot-with-cartopy/


If you use this in a scientific paper, we will appreciate a citation to the paper:

Omira, R., Ramalho, R.S., Kim, J., Gonzalez, P.J., Kadri, U., Miranda, J.M., Carrilho, F., Baptista, M.A., (2022) Global Tonga tsunami explained by a fast-moving atmospheric source. *Nature*, doi:[10.1038/s41586-022-04926-4](https://doi.org/10.1038/s41586-022-04926-4).

In [ ]:
# Required modules
from netCDF4 import Dataset                # Read / Write NetCDF4 files
import matplotlib.pyplot as plt            # Plotting library
import numpy as np                         # Scientific computing with Python
import cartopy, cartopy.crs as ccrs        # Plot maps
from datetime import datetime, timedelta   # Library to convert julian day to dd-mm-yyyy
import glob

files=glob.glob("OR_ABI-L2-CMIPF-M6C13_G16_s*")
for i in range(len(files)-1):
    fechahora1=files[i+1][27:39]
    fechahora2=files[i][27:39]
    
    # Open the GOES-16 image
    file1 = Dataset("OR_ABI-L2-CMIPF-M6C13_G16_s"+fechahora1+".nc")
    file1b = Dataset("OR_ABI-L2-CMIPF-M6C13_G16_s"+fechahora2+".nc")
    # Get the pixel values
    #data1 = file1.variables['CMI'][:,:] - 273.15
    data1 = (file1.variables['CMI'][:,:] - 273.15) - (file1b.variables['CMI'][:,:] - 273.15)
    # The projection x and y coordinates equals the scanning angle (in radians) multiplied by the satellite height
    sat_h = file1.variables['goes_imager_projection'].perspective_point_height
    x1 = file1.variables['x'][:] * sat_h
    y1 = file1.variables['y'][:] * sat_h
    # Define the image extent
    img_extent_1 = (x1.min(), x1.max(), y1.min(), y1.max())
     
    # Open the GOES-17 image
    file2 = Dataset("OR_ABI-L2-CMIPF-M6C13_G17_s"+fechahora1+".nc")
    file2b = Dataset("OR_ABI-L2-CMIPF-M6C13_G17_s"+fechahora2+".nc")
    # Get the pixel values
    #data2 = file2.variables['CMI'][:,0:4000] - 273.15
    data2 = (file2.variables['CMI'][:,0:4000] - 273.15) - (file2b.variables['CMI'][:,0:4000] - 273.15)
    # The projection x and y coordinates equals the scanning angle (in radians) multiplied by the satellite height
    sat_h = file2.variables['goes_imager_projection'].perspective_point_height
    x2 = file2.variables['x'][0:4000] * sat_h
    y2 = file2.variables['y'][:] * sat_h
    # Define the image extent
    img_extent_2 = (x2.min(), x2.max(), y2.min(), y2.max())
 
    # Choose the plot size (width x height, in inches)
    #plt.figure(figsize=(7,7))
    cm = 1/2.54  # centimeters in inches
    plt.figure(figsize=(30*cm, 30*cm)) # in cm

    # Plot definition
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=-103.3))#, globe=globe))
    ax.set_extent([-222.0, 10.0, -90.0, 90.0], crs=ccrs.PlateCarree())
    # Add a background image
    ax.stock_img()
    # Add coastlines, borders and gridlines
    ax.coastlines(resolution='10m', color='black', linewidth=0.8)
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.gridlines(color='gray', alpha=0.5, linestyle='--', linewidth=0.5)
    # Plot the image (cmap='magma' cmap='Greys' -2:2 and -5:5)
    img = ax.imshow(data1, vmin=-1, vmax=1, origin='upper', extent=img_extent_1, cmap='magma', transform=ccrs.Geostationary(central_longitude=-75.0, satellite_height=35786023.0))
    img = ax.imshow(data2, vmin=-1, vmax=1, origin='upper', extent=img_extent_2, cmap='magma', transform=ccrs.Geostationary(central_longitude=-137.0, satellite_height=35786023.0))     
    # Add a colorbar
    plt.colorbar(img, label='Brightness Temperatures (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)
 
    # Getting the file date
    add_seconds = int(file2.variables['time_bounds'][0])
    date = datetime(2000,1,1,12) + timedelta(seconds=add_seconds)
    date = date.strftime('%d %B %Y %H:%M UTC')
 
    # Add a title
    plt.title('GOES-16 and GOES-17 Band 13', fontweight='bold', fontsize=10, loc='left')
    plt.title('Full Disk \n' + date, fontsize=10, loc='right')
     
    # Save the image
    plt.savefig('Image_G16_G17_'+fechahora1+'_'+fechahora2+'.png',transparent=False)
    # Show the image
    plt.show()